In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Import and rearrange data

In [2]:
# Load data
bkgd = pd.read_hdf("data/preprocessed/bkgd.h5", key="bkgd")
vlq = pd.read_hdf("data/preprocessed/vlq.h5", key="vlq")
X_train = pd.concat([bkgd, vlq])
del bkgd, vlq

X_train

,FatJet_Multi,FatJet1_PT,FatJet2_PT,FatJet3_PT,FatJet4_PT,FatJet5_PT,FatJet1_Mass,FatJet2_Mass,FatJet3_Mass,FatJet4_Mass,...,Electron1_Eta,Electron2_Eta,Electron1_Phi,Electron2_Phi,MissingET_MET,MissingET_Phi,ScalarHT_HT,gen_weights,Label,Sample
6,1,222.386703,0.000000,0.000000,0.000000,0.000000,89.281700,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,74.745476,-2.622154,646.912720,1.916818e-06,0,ttbarZ_2L
21,1,355.976532,0.000000,0.000000,0.000000,0.000000,92.403839,0.000000,0.000000,0.000000,...,1.303101,1.311433,-3.124268,2.662745,62.061245,0.126687,738.070923,1.916818e-06,0,ttbarZ_2L
22,1,379.277893,0.000000,0.000000,0.000000,0.000000,92.015945,0.000000,0.000000,0.000000,...,-0.443531,0.041256,0.257705,0.629130,154.406555,2.711888,1017.925659,1.916818e-06,0,ttbarZ_2L
27,2,315.526428,210.032394,0.000000,0.000000,0.000000,79.493019,69.408371,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,88.304909,1.464204,784.078552,1.916818e-06,0,ttbarZ_2L
28,2,339.583130,219.172821,0.000000,0.000000,0.000000,116.564987,78.811958,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,99.853302,0.038497,775.420654,1.916818e-06,0,ttbarZ_2L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249910,4,666.686401,607.264709,450.049072,391.603912,0.000000,115.184601,179.727097,114.864006,64.223465,...,0.000000,0.000000,0.000000,0.000000,112.573494,-2.934912,2303.999756,1.774938e-07,1,mch45_HG_13TeV_wohg_HQ1400_test
249918,4,928.090820,700.795898,401.137665,232.056839,0.000000,346.194672,102.790993,94.033356,23.929482,...,0.000000,0.000000,0.000000,0.000000,365.781525,-2.217049,2508.777588,1.774938e-07,1,mch45_HG_13TeV_wohg_HQ1400_test
249942,2,1385.916504,1031.829224,0.000000,0.000000,0.000000,139.179977,158.596802,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,226.095474,-0.110796,2874.203857,1.774938e-07,1,mch45_HG_13TeV_wohg_HQ1400_test
249974,5,967.123413,555.906433,533.325867,327.264191,263.725922,173.991745,96.630959,131.944000,63.603821,...,0.000000,0.000000,0.000000,0.000000,329.906006,0.181167,2831.758057,1.774938e-07,1,mch45_HG_13TeV_wohg_HQ1400_test


In [3]:
# Train, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_train.drop(["Label"], axis=1), X_train["Label"], 
                                                    test_size=1/3, random_state=56)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.5, random_state=60)

In [4]:
# Save train data
X_train.to_hdf("data/classifier/train.h5", key="X")
y_train.to_hdf("data/classifier/train.h5", key="y")

# Save validation data
X_val.to_hdf("data/classifier/validation.h5", key="X")
y_val.to_hdf("data/classifier/validation.h5", key="y")

# Save test data
X_test.to_hdf("data/classifier/test.h5", key="X")
y_test.to_hdf("data/classifier/test.h5", key="y")

In [5]:
# Get data samples
train_samples, val_samples, test_samples = X_train["Sample"], X_val["Sample"], X_test["Sample"]

# Get data weights
train_weights, val_weights, test_weights = X_train["gen_weights"], X_val["gen_weights"], X_test["gen_weights"]

# Remove sample and weight columns
X_train.drop(["Sample", "gen_weights"], axis=1, inplace=True)
X_val.drop(["Sample", "gen_weights"], axis=1, inplace=True)
X_test.drop(["Sample", "gen_weights"], axis=1, inplace=True)

# Calculate class weights
class_weights = {
    0: 1,
    1: len(y_train[y_train==0]) / len(y_train[y_train==1])
}

# Create model

In [27]:
class Standardization(keras.layers.Layer):
    def adapt(self, data_samples):
        self.means = np.mean(data_samples, axis=0, keepdims=True)
        self.stds = np.std(data_samples, axis=0, keepdims=True)
        
    def call(self, inputs):
        return (inputs - self.means) / (self.stds + keras.backend.epsilon())
    
std_layer = Standardization()
std_layer.adapt(X_train.values)

In [28]:
inputs = keras.Input(shape=(69,))
std_inputs = std_layer(inputs)
fc1 = keras.layers.Dense(80, activation="relu")(std_inputs)
d1 = keras.layers.Dropout(0.3)(fc1, training=True)
fc2 = keras.layers.Dense(50, activation="relu")(d1)
d2 = keras.layers.Dropout(0.3)(fc2, training=True)
fc3 = keras.layers.Dense(30, activation="relu")(d2)
outputs = keras.layers.Dense(1, activation="sigmoid")(fc3)

model = keras.Model(inputs, outputs)
model.compile(optimizer=keras.optimizers.Adam(lr=1e-4), loss="binary_crossentropy", 
              metrics=["accuracy", keras.metrics.AUC()])

model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 69)]              0         
_________________________________________________________________
standardization_4 (Standardi (None, 69)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 80)                5600      
_________________________________________________________________
dropout_9 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 50)                4050      
_________________________________________________________________
dropout_10 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 30)               

In [29]:
# Model name
name = "Hidden:80, 50, 30|BatchS:256|Dropout:0.3"

# Callbacks

In [30]:
# Tensorboard
TB = keras.callbacks.TensorBoard("logs/" + name, write_images=True)

# Early Stopping
ES = keras.callbacks.EarlyStopping(monitor="val_loss", patience=30, verbose=2, mode="min")

# Model Checkpoint
MC = keras.callbacks.ModelCheckpoint("models/" + name + ".h5", save_best_only=True, monitor="val_loss",
                                     mode="min")

# Reduce LR on Plateau
LR = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=6, mode="min", 
                                       min_lr=1e-6)

# Training the Model

In [31]:
model.fit(X_train.values, y_train.values, batch_size=256, epochs=500, callbacks=[TB, ES, MC, LR],
          validation_data=(X_val.values, y_val.values, val_weights.values), shuffle=True,
          sample_weight=train_weights.values, class_weight=class_weights)

Epoch 1/500
1078/1078 [==============================] - 15s 14ms/step - loss: 5.9939e-05 - accuracy: 0.7270 - auc_4: 0.6193 - val_loss: 3.1624e-06 - val_accuracy: 0.7403 - val_auc_4: 0.7572
Epoch 2/500
1078/1078 [==============================] - 3s 3ms/step - loss: 3.2481e-06 - accuracy: 0.7403 - auc_4: 0.7828 - val_loss: 1.5041e-06 - val_accuracy: 0.7403 - val_auc_4: 0.8135
Epoch 3/500
1078/1078 [==============================] - 3s 3ms/step - loss: 2.4448e-06 - accuracy: 0.7403 - auc_4: 0.8474 - val_loss: 1.1557e-06 - val_accuracy: 0.7403 - val_auc_4: 0.8739
Epoch 4/500
1078/1078 [==============================] - 3s 3ms/step - loss: 2.1059e-06 - accuracy: 0.7405 - auc_4: 0.8924 - val_loss: 9.9042e-07 - val_accuracy: 0.7405 - val_auc_4: 0.9052
Epoch 5/500
1078/1078 [==============================] - 3s 3ms/step - loss: 1.9034e-06 - accuracy: 0.7410 - auc_4: 0.9156 - val_loss: 8.7901e-07 - val_accuracy: 0.7413 - val_auc_4: 0.9225
Epoch 6/500
1078/1078 [==============================